In [ ]:
import pyam
import nomenclature
import ixmp

In [ ]:
scenario = "GECO_Ref_AR6"

In [ ]:
df = pyam.IamDataFrame(f"results_{scenario}_20230206.xlsx")

In [ ]:
df_eu27 = pyam.IamDataFrame(f"results_{scenario}_20230202.xlsx")

In [ ]:
df_eu27 = df_eu27.filter(region="EU28 MAP").rename(region={"EU28 MAP": "EU27"})

In [ ]:
df.append(df_eu27, inplace=True)

In [ ]:
df.rename(model={"POLES ENGAGE": "POLES-JRC"}, inplace=True)

In [ ]:
df.rename(scenario={"CurPol": "Current Policies"}, inplace=True)

In [ ]:
poles_rename_mapping = {
    "AFMI MAP": "POLES-JRC|Africa & Middle East",
    "ARG MAP": "POLES-JRC|Argentina",
    "ASIA MAP": "POLES-JRC|Asia (excl. Japan)",
    "AUS MAP": "POLES-JRC|Australia",
    "BRA MAP": "POLES-JRC|Brazil",
    "CAN MAP": "POLES-JRC|Canada",
    "CHN MAP": "POLES-JRC|China",
    "IDN MAP": "POLES-JRC|Indonesia",
    "IND MAP": "POLES-JRC|India",
    "JPN MAP": "POLES-JRC|Japan",
    "KOR MAP": "POLES-JRC|South Korea",
    "LAM MAP": "POLES-JRC|Latin America",
    "MEX MAP": "POLES-JRC|Mexico",
    "REF MAP": "POLES-JRC|Former CIS",
    "RUS MAP": "POLES-JRC|Russian Federation",
    "SAU MAP": "POLES-JRC|Saudi Arabia",
    "TUR MAP": "POLES-JRC|Turkey",
    "USA MAP": "POLES-JRC|USA",
    "ZAF MAP": "POLES-JRC|South Africa",
    "WRD MAP": "World",
    "EU28 MAP": "EU27 & UK",
    "OECD90EU MAP": "OECD & EU (R5)",
    "R10AFRICA MAP": "Africa (R10)",
    "R10CHINAplus MAP": "China+ (R10)",
    "R10EUROPE MAP": "Europe (R10)",
    "R10INDIAplus MAP": "India+ (R10)",
    "R10LATINAM MAP": "Latin America (R10)",
    "R10MIDDLEEAST MAP": "Middle East (R10)",
    "R10NORTHAM MAP": "North America (R10)",
    "R10PACOECD MAP": "Pacific OECD (R10)",
    "R10REFECON MAP": "Reforming Economies (R10)",
    "R10RESTASIA MAP": "Rest of Asia (R10)",
    "R10ROWO MAP": "Other (R10)",
}
df.rename(region=poles_rename_mapping, inplace=True)

In [ ]:
df.filter(region=["THA MAP", "VNM MAP"], keep=False, inplace=True)

In [ ]:
eu_climate_dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
df_eu = df.filter(region=["EU27 & UK", "EU27"])

In [ ]:
df_eu.filter(variable="Primary Energy").timeseries()

In [ ]:
unit_rename_mapping = {
    "billion US$2010/yr OR local currency": "billion US$2010/yr",
    "US$2010/kW OR local currency/kW": "US$2010/kW",
    "US$2010/kW/yr OR local currency/kW/yr": "US$2010/kW/yr",
    "billion US$2010/yr or local currency/yr": "billion US$2010/yr",
    "billion US$2010/yr OR local currency/yr": "billion US$2010/yr",
    "US$2010/GJ or local currency/GJ": "US$2010/GJ",
    "US$2010/t CO2 or local currency/t CO2": "US$2010/t CO2",
    "bn tkm/yr": "billion tkm/yr",
    "bn pkm/yr": "billion pkm/yr",
}

df.rename(unit=unit_rename_mapping, inplace=True)

In [ ]:
df.filter(variable="Price|Agriculture|Non-Energy Crops and Livestock|Index", keep=False, inplace=True)

In [ ]:
df.filter(variable=eu_climate_dsd.variable, inplace=True)

In [ ]:
eu_climate_dsd.validate(df)

In [ ]:
df

In [ ]:
eu_climate = ixmp.Platform("eu-climate-submission")

In [ ]:
for (_model, _scenario) in df.index:
    _df = df.filter(model=_model, scenario=_scenario)

    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(eu_climate, _model, f"GECO-2023 {_scenario}", version="new")
    scen.add_timeseries(_df.data)
    scen.commit("Imported from GECO 2023")
    scen.set_as_default()

    scen.set_meta({"Source": "GECO 2023"})